# Rotations

## Goal 
- The goal of this code is to develop rotation matrices/tensors for dealing with the following:
- rotate particle positions into parallel and perpendicular shear direction
- use rotated particle positions to compute strain applied to boundary
- rotate strain measures into parallel and perp directions
    - compute strain with rotated particle positions
    - rotate strain computed in image coordinates using (essentialy) mohr circle
    - explicitly check that these strain measures are identical. 
- diagonalize the local strain tensor everywhere 
- compute the rotation vector between two locally diagonal strain tensors (useful for core regions)

In [36]:
# preamble 
import numpy as np
import dask.array as da
import numba

import sys
sys.path.extend(['/Users/zsolt/Colloid_git/TractionRheoscopy'])
from data_analysis import eshelby_inclusion as slb
from data_analysis import grid_track, rotation
import tifffile
import pandas as pd
import dask
import yaml

In [2]:
# load small data from Aidan. Just two time points
def loadParticle(t, path_partial = None, fName_frmt = None):
    if path_partial is None: 
        path_partial = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/locations_stitch/partial_data_Aidan/'
    if fName_frmt is None: 
        fName_frmt = 'shearRun10292018f_centerParticles_t{:02}.h5'
    
    pos_t = pd.read_hdf(path_partial + fName_frmt.format(t))
    idx_t = pd.MultiIndex.from_product([[t],pos_t.index], names = ['frame', 'particle'] )
    
    return pos_t.set_index(idx_t)

#path_partial = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/locations_stitch/partial_data_Aidan/'
#fName_frmt = 'shearRun10292018f_centerParticles_t{:02}.h5'

posList = []
for t in range(3):
    posList.append(loadParticle(t))
pos = pd.concat(posList)
    

In [3]:
pos

x (um, imageStack)  y (um, imageStack)  z (um, imageStack)  \
frame particle                                                               
0     74955              90.492940           76.764406           27.902609   
      74956              91.195593           78.102822           27.900874   
      75057              91.450079           75.565372           28.032157   
      75063              93.676084           76.267803           27.864490   
      75068              92.161994           76.933751           27.942663   
...                            ...                 ...                 ...   
2     976684            138.401187          139.865731           83.545205   
      976727            148.561716          148.372527           83.414681   
      976749            144.581477          145.052961           84.499723   
      976750            145.830648          145.595963           84.431944   
      976776            143.208101          145.089160           85.342912   

                   x_std     y_std     z_std  
frame particle                                
0     74955     0.134505  0.128758  0.144804  
      74956     0.142430  0.140001  0.138219  
      75057     0.115519  0.117071  0.145252  
      75063     0.125139  0.122297  0.134428  
      75068     0.113436  0.112211  0.117256  
...                  ...       ...       ...  
2     976684    0.267231       NaN  0.540731  
      976727    1.411727  1.353646  3.239737  
      976749         NaN       NaN       NaN  
      976750         NaN       NaN       NaN  
      976776         NaN       NaN       NaN  

[470123 rows x 6 columns]

In [4]:
# compute the strain in the usual coordinate
from data_analysis import static
def computeStrain(pos_df, tPair, output = 'strainTraj', pos_keys=None, verbose=False):
    """
    Make LocalStrainTraj on a list of time points
    tPairs = list(zip([0 for n in range(90)],[n for n in range(2,90)]))

    This is a wrapper.  Mostly handles formatting the dataFrames.
    """ 
    ref = tPair[0]
    cur = tPair[1]
    strain_traj = static.localStrain(pos_df,ref,cur,pos_keys=pos_keys)
    #strain_traj = localStrain(pos_df, 0, 1, pos_keys = pos_keys)
    strain_traj = strain_traj.stack().rename('({},{})'.format(ref,cur)).to_frame()
    strain_traj.set_index(strain_traj.index.rename(['particle', 'values']), inplace=True)
    if output == 'hdf':
        raise KeyError('Saving strainTraj directly to hdf is not implemented yet')
        #strain_fName = 'tfrGel10212018A_shearRun10292018f_sed_strainTraj_consecutive.h5'
        #strain_traj.to_hdf(hdf_stem + strain_fName, '(0,t)', mode='a', format='table', data_columns=True)
    elif output == 'strainTraj':
        return strain_traj
    else: raise KeyError('output {} not recognized'.format(output))

#tPairs = [(0,1),(0,2)]
#subPosList = [pos.loc[([t[0],t[1]],slice(None)),:][['x (um, imageStack)','y (um, imageStack)', 'z (um, imageStack)']] for t in tPairs]
strain_traj_um_imageStack = computeStrain(pos,(0,1))

In [5]:
strain_traj

(0,1)
particle values              
74955    D2_min      0.125524
         exx        -0.004519
         exy        -0.001776
         exz        -0.020201
         eyy         0.005996
...                       ...
890420   ezz         0.004417
         rxy        -0.007035
         rxz        -0.013433
         ryz        -0.003190
         nnb count  10.000000

[1713096 rows x 1 columns]

In [6]:
# read rotation parameters from file and construct rotation matrix
def parseRotYaml(rotDict):
    """ From an opened yaml file giving rotation values, return rotation matrices"""
    if rotDict['handed'] == 'right' and rotDict['positiveSignature'] == 'clockwise':
        theta_x, theta_y, theta_z = rotDict['theta_x'],rotDict['theta_y'], rotDict['theta_z']
        r_x = np.array(((1,0,0),
                        (0, np.cos(theta_x), -1*np.sin(theta_x)),
                        (0, np.sin(theta_x), np.cos(theta_x))))
        r_y = np.array(((np.cos(theta_y), 0, np.sin(theta_y)),
                        (0, 1, 0),
                        (-1*np.sin(theta_y), 0, np.cos(theta_y))))
        r_z = np.array(((np.cos(theta_z), -1*np.sin(theta_z), 0),
                        (np.sin(theta_z), np.cos(theta_z), 0),
                        (0, 0, 1)))
        prod_zyx = np.matmul(r_z,np.matmul(r_y,r_x))
        prod_xyz = np.matmul(r_x.T,np.matmul(r_y.T, r_z.T))
        return {'prod_zyx (left)': prod_zyx, '(prod_zyx)T (right)': prod_xyz, 'r_x': r_x, 'r_y': r_y, 'r_z': r_z}
    else: raise ValueError("not all options are supported in parseRotYaml yet...")
        
def coordTransform(pos_df, coordStr_current, coordStr_target,**kwargs):
    if coordStr_current == '(um, imageStack)' and coordStr_target == '(um, rheo_sedHeight)':
        z_offSet = kwargs['z_offSet']
        pos_df['x {}'.format(coordStr_target)] = pos_df['x {}'.format(coordStr_current)] - 235/2.0
        pos_df['y {}'.format(coordStr_target)] = -1*pos_df['y {}'.format(coordStr_current)] + 235/2.0
        pos_df['z {}'.format(coordStr_target)] = pos_df['z {}'.format(coordStr_current)] + z_offSet
        return pos_df
    else: raise  ValueError('Coordinate transofrm between {} and {} not recognized or not yet supported'.format(coordStr_current, coordStr_target))

In [7]:
with open('rotation.yaml') as f:
    rotDict = yaml.load(f, Loader=yaml.FullLoader)
rotDict
r = parseRotYaml(rotDict['rotationMatrix'])['prod_zyx (left)']

In [8]:
coordTransform(pos,'(um, imageStack)', '(um, rheo_sedHeight)',z_offSet = 0)
pos

x (um, imageStack)  y (um, imageStack)  z (um, imageStack)  \
frame particle                                                               
0     74955              90.492940           76.764406           27.902609   
      74956              91.195593           78.102822           27.900874   
      75057              91.450079           75.565372           28.032157   
      75063              93.676084           76.267803           27.864490   
      75068              92.161994           76.933751           27.942663   
...                            ...                 ...                 ...   
2     976684            138.401187          139.865731           83.545205   
      976727            148.561716          148.372527           83.414681   
      976749            144.581477          145.052961           84.499723   
      976750            145.830648          145.595963           84.431944   
      976776            143.208101          145.089160           85.342912   

                   x_std     y_std     z_std  x (um, rheo_sedHeight)  \
frame particle                                                         
0     74955     0.134505  0.128758  0.144804              -27.007060   
      74956     0.142430  0.140001  0.138219              -26.304407   
      75057     0.115519  0.117071  0.145252              -26.049921   
      75063     0.125139  0.122297  0.134428              -23.823916   
      75068     0.113436  0.112211  0.117256              -25.338006   
...                  ...       ...       ...                     ...   
2     976684    0.267231       NaN  0.540731               20.901187   
      976727    1.411727  1.353646  3.239737               31.061716   
      976749         NaN       NaN       NaN               27.081477   
      976750         NaN       NaN       NaN               28.330648   
      976776         NaN       NaN       NaN               25.708101   

                y (um, rheo_sedHeight)  z (um, rheo_sedHeight)  
frame particle                                                  
0     74955                  40.735594               27.902609  
      74956                  39.397178               27.900874  
      75057                  41.934628               28.032157  
      75063                  41.232197               27.864490  
      75068                  40.566249               27.942663  
...                                ...                     ...  
2     976684                -22.365731               83.545205  
      976727                -30.872527               83.414681  
      976749                -27.552961               84.499723  
      976750                -28.095963               84.431944  
      976776                -27.589160               85.342912  

[470123 rows x 9 columns]

In [9]:
# write a function that rotates particle positions
# write a function that rotates strain
# in both cases, the difficult part is formatting the dataframe so that the matrix multiplication can
# be broadcast in numpy. 
# I think the best way forward to wrap the function and create a numpy array, apply braodcast in numpy, and 
# then wrap back into dataframe with the correct index labels. 
# I feel like I have to do this type of operation all the time. Maybe there is a way to make the code reusable?

def rotatePosition(df, rotMatrix, posKeys=None):
    # get numpy array of particle positions to rotate
    # maybe also drop na ?
    # reformat and resize
    # check that the ordering of the coordinates on the position array matches the rotation matrix
    # ...should default to zyx
    if posKeys is None:
        suffix = '(um, rheo_sedHeight)'
        posKeys = {'x': 'x {}'.format(suffix), 'y':'y {}'.format(suffix),'z': 'z {}'.format(suffix)}
    pos = df[posKeys.values()].to_numpy().T
    idx = df[posKeys.values()].index
    
    # this likely needs to be a array shape so that the matrix multiplication can be broadcast correctly
    rotPos = (rotMatrix @ pos).T
    
    # set index and probably restack
    # I think we'll need a join operation on the dataFrame
    # convert to dataFrame with the right index 
    rotPos_df = pd.DataFrame(data = rotPos, index=idx, columns=posKeys.values())
    
    # should I return a modfied dataframe or just the section of the dataframe that can be joined to the 
    # input dataFrame? How about just the section I computed...not the whole thing
    return rotPos_df

In [14]:
suffix = '(um, rheo_sedHeight)'
rot_keys = {'x': 'x {}'.format(suffix), 'y':'y {}'.format(suffix), 'z':'z {}'.format(suffix)}
rot_keys
r

array([[ 0.40808206, -0.91294525,  0.        ],
       [ 0.91294525,  0.40808206,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [15]:
microPos = pos.head(5)[rot_keys.values()]
idx = microPos.index
pos_np = microPos.to_numpy().T
rot_pos = (r @ pos_np).T
rot_pos_df = pd.DataFrame(data = rot_pos,index=idx,columns=rot_keys.values())

In [16]:
rotatePosition(pos,r)

x (um, rheo_sedHeight)  y (um, rheo_sedHeight)  \
frame particle                                                   
0     74955                 -48.210464               -8.032502   
      74956                 -46.701823               -7.937202   
      75057                 -48.914525               -6.669383   
      75063                 -47.364851               -4.923811   
      75068                 -47.374750               -6.577853   
...                                ...                     ...   
2     976684                 28.948087                9.954586   
      976727                 40.860656               15.759122   
      976749                 36.205810               13.480037   
      976750                 37.211305               14.398872   
      976776                 35.678408               12.211447   

                z (um, rheo_sedHeight)  
frame particle                          
0     74955                  27.902609  
      74956                  27.900874  
      75057                  28.032157  
      75063                  27.864490  
      75068                  27.942663  
...                                ...  
2     976684                 83.545205  
      976727                 83.414681  
      976749                 84.499723  
      976750                 84.431944  
      976776                 85.342912  

[470123 rows x 3 columns]

In [17]:
rot_pos_df

x (um, rheo_sedHeight)  y (um, rheo_sedHeight)  \
frame particle                                                   
0     74955                 -48.210464               -8.032502   
      74956                 -46.701823               -7.937202   
      75057                 -48.914525               -6.669383   
      75063                 -47.364851               -4.923811   
      75068                 -47.374750               -6.577853   

                z (um, rheo_sedHeight)  
frame particle                          
0     74955                  27.902609  
      74956                  27.900874  
      75057                  28.032157  
      75063                  27.864490  
      75068                  27.942663

In [18]:
r

array([[ 0.40808206, -0.91294525,  0.        ],
       [ 0.91294525,  0.40808206,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [19]:
rotDict

{'rotationMatrix': {'coordStr': '(um, rheo_sedHeight)',
  'handed': 'right',
  'positiveSignature': 'clockwise',
  'units': 'degrees',
  'theta_x': 0,
  'theta_y': 0,
  'theta_z': 20}}

In [20]:
r_left = parseRotYaml(rotDict['rotationMatrix'])['prod_zyx (left)']
r_right = parseRotYaml(rotDict['rotationMatrix'])['(prod_zyx)T (right)']
#(r_right @ (r_left @ microPos[rot_keys.values()].to_numpy().T)).T
# should give the identity (TRUE)
r_left @ r_right
#microPos[rot_keys.values()].to_numpy() @ r_right
#((r_left @ microPos[rot_keys.values()].to_numpy().T).T @ r_right.T)
#(r_right @ microPos[rot_keys.values()].to_numpy().T).T @ r_left 
#(r_left @ microPos[rot_keys.values()].to_numpy()[0].T) @ r_right

array([[ 1.00000000e+00, -1.33281488e-17,  0.00000000e+00],
       [-1.33281488e-17,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [21]:
r_right.T

array([[ 0.40808206, -0.91294525,  0.        ],
       [ 0.91294525,  0.40808206,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [22]:
r_left

array([[ 0.40808206, -0.91294525,  0.        ],
       [ 0.91294525,  0.40808206,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [23]:
# apply left rotation
v = microPos[rot_keys.values()].to_numpy().T
r_right.dot((r_left.dot(v))).T
(r_left.dot(v)).T.dot(r_left)
(r_left @ v).T @ r_left
(r_right @ (r_left @ v)).T

array([[-27.0070603 ,  40.7355938 ,  27.90260905],
       [-26.30440729,  39.39717771,  27.90087441],
       [-26.04992146,  41.93462773,  28.03215656],
       [-23.82391555,  41.23219689,  27.86448974],
       [-25.33800554,  40.56624913,  27.94266273]])

{'x': 'x (um, rheo_sedHeight)',
 'y': 'y (um, rheo_sedHeight)',
 'z': 'z (um, rheo_sedHeight)'}

# Compare the strains due to coordinate transformation (um, imageStack) -> (um, rheo_sedHeight)
Results:
- D2_min changes value
- exy, eyz change sign..all other are unchanged due to y-> -y


In [30]:
# rotate the particle locations and recompute the strains
# Not quite...this isjust a coordinate transform, not a rotation of the particle positions. 
strainTraj_rheoSedDept = computeStrain(pos,(0,1),pos_keys=rot_keys)
rot_keys

{'x': 'x (um, rheo_sedHeight)',
 'y': 'y (um, rheo_sedHeight)',
 'z': 'z (um, rheo_sedHeight)'}

In [28]:
strainTraj_rheoSedDept.head(10)

(0,1)
particle values          
74955    D2_min  0.118952
         exx    -0.004519
         exy     0.001776
         exz    -0.020201
         eyy     0.005996
         eyz    -0.003453
         ezz    -0.041497
         rxy    -0.002239
         rxz     0.011572
         ryz    -0.001985

In [29]:
strain_traj_um_imageStack.head(10)

(0,1)
particle values          
74955    D2_min  0.125524
         exx    -0.004519
         exy    -0.001776
         exz    -0.020201
         eyy     0.005996
         eyz     0.003453
         ezz    -0.041497
         rxy     0.002239
         rxz     0.011572
         ryz     0.001985

# Check what happens when position is rotated and strain is computed

## rotate pos

In [45]:
r, rot_keys

(array([[ 0.40808206, -0.91294525,  0.        ],
        [ 0.91294525,  0.40808206,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 {'x': 'x (um, rheo_sedHeight)',
  'y': 'y (um, rheo_sedHeight)',
  'z': 'z (um, rheo_sedHeight)'})

In [44]:
pos_rot_r = rotation.rotatePosition(pos,r)
pos
pos_rot_r

x (um, rheo_sedHeight)  y (um, rheo_sedHeight)  \
frame particle                                                   
0     74955                 -48.210464               -8.032502   
      74956                 -46.701823               -7.937202   
      75057                 -48.914525               -6.669383   
      75063                 -47.364851               -4.923811   
      75068                 -47.374750               -6.577853   
...                                ...                     ...   
2     976684                 28.948087                9.954586   
      976727                 40.860656               15.759122   
      976749                 36.205810               13.480037   
      976750                 37.211305               14.398872   
      976776                 35.678408               12.211447   

                z (um, rheo_sedHeight)  
frame particle                          
0     74955                  27.902609  
      74956                  27.900874  
      75057                  28.032157  
      75063                  27.864490  
      75068                  27.942663  
...                                ...  
2     976684                 83.545205  
      976727                 83.414681  
      976749                 84.499723  
      976750                 84.431944  
      976776                 85.342912  

[470123 rows x 3 columns]

In [46]:
## compute strain on rotated position
strainTraj_rheoSedDept_rot = computeStrain(pos_rot_r,(0,1),pos_keys=rot_keys)
rot_keys

{'x': 'x (um, rheo_sedHeight)',
 'y': 'y (um, rheo_sedHeight)',
 'z': 'z (um, rheo_sedHeight)'}

In [25]:
strainTraj_rot.loc[(slice(None),['exx','eyy','ezz']),:]
strainTraj_rot.loc[(74955,['exx','eyy','ezz']),:]

NameError: name 'strainTraj_rot' is not defined

In [47]:
strainTraj_rheoSedDept_rot

(0,1)
particle values              
74955    D2_min      0.120679
         exx         0.002922
         exy        -0.005102
         exz        -0.005092
         eyy        -0.001444
...                       ...
890420   ezz         0.004417
         rxy         0.007035
         rxz        -0.008394
         ryz        -0.010962
         nnb count  10.000000

[1713096 rows x 1 columns]

## Now compare strainTraj_rheoSedDept_rot to strain rotation applied straintraj_rheoSedDepth

### This the strain compute in rheo_sedHeight with NO ROTATION

In [49]:
# convert format of array to feed into _rotate
# convert trajectory format to frameParticle format
sedStrain_tmp = strainTraj_rheoSedDept.transpose().stack('particle')
idx = sedStrain_tmp.index.set_names(['frame','particle'])
sedStrain_rheoSedDept_frameParticle = pd.DataFrame(sedStrain_tmp, index = idx)
sedStrain_rheoSedDept_frameParticle

values             D2_min       exx       exy       exz       eyy       eyz  \
frame particle                                                                
(0,1) 74955      0.118952 -0.004519  0.001776 -0.020201  0.005996 -0.003453   
      74956      0.179941  0.005630  0.000683 -0.015843  0.001288  0.003656   
      75057      0.145680  0.006341 -0.004115  0.020021 -0.000549 -0.000321   
      75063      0.135813 -0.005571  0.000750 -0.004846 -0.004481  0.009011   
      75068      0.109150  0.003133 -0.003027  0.003807  0.003518  0.007359   
...                   ...       ...       ...       ...       ...       ...   
      890365    11.906913 -0.023054  0.043684 -0.200029 -0.069195  0.171321   
      890366     0.171452  0.017742  0.005746  0.009057 -0.013608  0.005041   
      890399     0.488662  0.002738  0.002893 -0.005607 -0.004916 -0.000156   
      890413     0.050888  0.001162  0.011369 -0.001567  0.008637  0.000204   
      890420     0.184030  0.001207  0.000007  0.022489 -0.000981 -0.025998   

values               ezz  nnb count       rxy       rxz       ryz  
frame particle                                                     
(0,1) 74955    -0.041497        9.0 -0.002239  0.011572 -0.001985  
      74956    -0.038488       10.0  0.001190  0.029955  0.006666  
      75057     0.040780       10.0 -0.003687  0.011841  0.007452  
      75063    -0.033722       10.0  0.002588  0.000266  0.012871  
      75068     0.014186       13.0  0.002215  0.000421 -0.004411  
...                  ...        ...       ...       ...       ...  
      890365    0.085185        8.0  0.033508 -0.213362  0.210185  
      890366    0.000765       12.0 -0.008108 -0.002670  0.016994  
      890399    0.030244       15.0  0.001310  0.004838 -0.007818  
      890413    0.016855        8.0 -0.004991  0.010913  0.001619  
      890420    0.004417       10.0  0.007035 -0.013433  0.003190  

[155736 rows x 11 columns]

### This is the strain compute by rotating previously calculated strain in rheo_sedHeight by 20 deg

In [63]:
j = np.array([[1,0,0],[0,-1,0],[0,0,1]])
r, r.T, r @ r.T, r-r.T, r@j@(r@j).T

(array([[ 0.40808206, -0.91294525,  0.        ],
        [ 0.91294525,  0.40808206,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.40808206,  0.91294525,  0.        ],
        [-0.91294525,  0.40808206,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 1.00000000e+00, -1.33281488e-17,  0.00000000e+00],
        [-1.33281488e-17,  1.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[ 0.       , -1.8258905,  0.       ],
        [ 1.8258905,  0.       ,  0.       ],
        [ 0.       ,  0.       ,  0.       ]]),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))

In [54]:
rot_keyList = ['exx','exy','exz','eyy','eyz','ezz']
sig_np = np.array([(0,0), (0,1), (0,2), (1,1), (1,2), (2,2)])
sedStrain_noRotate_np = sedStrain_rheoSedDept_frameParticle.xs('(0,1)',level='frame')[rot_keyList].to_numpy()
sedStrain_rot_np = rotation._rotate(sedStrain_noRotate_np,sig_np, r)
sedStrain_rotatedStrain = pd.DataFrame(data=sedStrain_rot_np,
                                       index=sedStrain_rheoSedDept_frameParticle.index, 
                                       columns= ['exx','exy','exz','eyy','eyz','ezz']).join(
    sedStrain_rheoSedDept_frameParticle[['nnb count','D2_min']])

In [55]:
sedStrain_rotatedStrain

exx       exy       exz       eyy       eyz       ezz  \
frame particle                                                               
(0,1) 74955     0.002922 -0.005102 -0.005092 -0.001444 -0.019852 -0.041497   
      74956     0.001502  0.001162 -0.009803  0.005416 -0.012971 -0.038488   
      75057     0.003665  0.005312  0.008463  0.002127  0.018147  0.040780   
      75063    -0.005222 -0.000906 -0.010204 -0.004830 -0.000747 -0.033722   
      75068     0.005709  0.001876 -0.005165  0.000942  0.006479  0.014186   
...                  ...       ...       ...       ...       ...       ...   
      890365   -0.094061 -0.011944 -0.238036  0.001812 -0.112703  0.085185   
      890366   -0.012668  0.007848 -0.000906  0.016803  0.010326  0.000765   
      890399   -0.005797  0.000922 -0.002146  0.003619 -0.005182  0.030244   
      890413   -0.001079 -0.010368 -0.000825  0.010878 -0.001347  0.016855   
      890420   -0.000622  0.000810  0.032912  0.000848  0.009922  0.004417   

                nnb count     D2_min  
frame particle                        
(0,1) 74955           9.0   0.118952  
      74956          10.0   0.179941  
      75057          10.0   0.145680  
      75063          10.0   0.135813  
      75068          13.0   0.109150  
...                   ...        ...  
      890365          8.0  11.906913  
      890366         12.0   0.171452  
      890399         15.0   0.488662  
      890413          8.0   0.050888  
      890420         10.0   0.184030  

[155736 rows x 8 columns]

### This is the strain that compute by first rotating the particle position by 20 deg and then compute the strain as usual

In [57]:
strainTraj_rheoSedDept_rot.transpose().stack('particle')[['exx','exy','exz','eyy','eyz','ezz','nnb count', 'D2_min']]

values               exx       exy       exz       eyy       eyz       ezz  \
      particle                                                               
(0,1) 74955     0.002922 -0.005102 -0.005092 -0.001444 -0.019852 -0.041497   
      74956     0.001502  0.001162 -0.009803  0.005416 -0.012971 -0.038488   
      75057     0.003665  0.005312  0.008463  0.002127  0.018147  0.040780   
      75063    -0.005222 -0.000906 -0.010204 -0.004830 -0.000747 -0.033722   
      75068     0.005709  0.001876 -0.005165  0.000942  0.006479  0.014186   
...                  ...       ...       ...       ...       ...       ...   
      890365   -0.094061 -0.011944 -0.238036  0.001812 -0.112703  0.085185   
      890366   -0.012668  0.007848 -0.000906  0.016803  0.010326  0.000765   
      890399   -0.005797  0.000922 -0.002146  0.003619 -0.005182  0.030244   
      890413   -0.001079 -0.010368 -0.000825  0.010878 -0.001347  0.016855   
      890420   -0.000622  0.000810  0.032912  0.000848  0.009922  0.004417   

values          nnb count     D2_min  
      particle                        
(0,1) 74955           9.0   0.120679  
      74956          10.0   0.175184  
      75057          10.0   0.144254  
      75063          10.0   0.099703  
      75068          13.0   0.110192  
...                   ...        ...  
      890365          8.0  11.793915  
      890366         12.0   0.168109  
      890399         15.0   0.509006  
      890413          8.0   0.047243  
      890420         10.0   0.200706  

[155736 rows x 8 columns]

In [31]:
# go from a strain traj to numpy array of strain tensors
def _makeStrainMatrix(strainList,signature=None):
    if signature is None or signature == 'upper triangle':
        signature = [(i,j) for i in range(3) for j in range(i,3)]

    out = np.zeros((3,3))
    for n in range(len(signature)):
        out[signature[n]] = strainList[n]

    out_sym = out + out.T - np.diag(np.diag(out))
    return out_sym

In [218]:
#signature = strain_traj.loc[809420]['(0,1)'].keys()
signature = 'upper triangle'
_sigList = ['exx','exy','exz','eyy','eyz','ezz']
strainList = strain_traj.loc[([80942,74955],_sigList),'(0,1)'].to_numpy()
strainList, _makeStrainMatrix(np.split(strainList,2)[0])
#strain_traj.loc[809420]['(0,1)'].keys(), strainList

(array([-0.00012559,  0.00110706, -0.00205914,  0.00135844, -0.00303321,
        -0.006647  , -0.00451882, -0.00177625, -0.02020148,  0.00599611,
         0.00345265, -0.04149744]),
 array([[-0.00012559,  0.00110706, -0.00205914],
        [ 0.00110706,  0.00135844, -0.00303321],
        [-0.00205914, -0.00303321, -0.006647  ]]))

In [234]:
tmp = np.array(np.split(strainList,strainList.shape[0]/6.))
tmp

array([[-0.00012559,  0.00110706, -0.00205914,  0.00135844, -0.00303321,
        -0.006647  ],
       [-0.00451882, -0.00177625, -0.02020148,  0.00599611,  0.00345265,
        -0.04149744]])

In [249]:
out = np.zeros(2)
out[:] = np.sum(tmp.T[:])
tmp[:], out, np.sum(tmp[0]), np.sum(tmp[1])

(array([[-0.00012559,  0.00110706, -0.00205914,  0.00135844, -0.00303321,
         -0.006647  ],
        [-0.00451882, -0.00177625, -0.02020148,  0.00599611,  0.00345265,
         -0.04149744]]),
 array([-0.06794469, -0.06794469]),
 -0.009399451229126873,
 -0.05854523533405162)

In [219]:

 
# initalize the output array to have dimensions:
#   - either (N,3,3) for storing strain arrays for each of the N particles
#   - flattened version to store only the upper diagonal
# split into the correct number of particles N using np split
# loop over the split list, compute the strain matrix each time by applying _makeStrainMatrix
# rotate by conjugating with rotation matrix
# reflatten (?)
# add to the output
# return the output
# deal with reformatting into pandas array with the correct index for particle number in an additional function. 

In [252]:
tmp = np.array(np.split(strain_traj.loc[([80942,74955],_sigList),'(0,1)'].to_numpy(),2))
out = np.zeros(2)
for n in range(tmp.shape[0]):
    out[n] = np.sum(tmp[n])
out

array([-0.00939945, -0.05854524])

In [250]:
from numba import jit

In [303]:
@jit(nopython = True)
def _rotate(strainList_np, signature_np, r):
    """
    Param
    :strainList_np: numpy array of strain components
        ->> strainList_np[10,:] gives [exx,exy,exz,eyy,eyz,ezz] of the 10th particle 
    :signature_np is a flattened version of the array coordinates of the upper triangle 
        -> signature_np = [(0,0), (0,1), (0,2), (1,1), (1,2), (2,2)]
    : r is the rotation matrix
    """
    
    # initialize output array
    out = np.zeros(strainList_np.shape)
    
    # loop over each particle
    for n in range(strainList_np.shape[0]):
        strain_flat = strainList_np[n]
        
        # init a matrix for rotation
        tmp = np.zeros((3,3))
        # this should be computed once and passed to the function
        #signature = [(i,j) for i in range(3) for j in range(i,3)]
        
        # loop over the components
        for comp in range(6):
            x = signature_np[comp][0]
            y = signature_np[comp][1]
            tmp[x,y] = strain_flat[comp]
        strain = tmp + tmp.T - np.diag(np.diag(tmp))
        #print(strain.shape)
        
        # now rotate the strain by conjugating with rot matrix
        strain_rot = r @ strain @ r.T
        #print(strain_rot)
        
        # flatten taking only the upper triangle
        strain_rot_flat = np.zeros(6)
        for comp in range(6):
            x = signature_np[comp][0]
            y = signature_np[comp][1]
            strain_rot_flat[comp] = strain_rot[x,y]
        
        # assign to output array
        out[n,:] = strain_rot_flat
    
    return out 

def rotateStrain(strain_df, time_str,r, signature=None):
    """
    Wrapper function on _rotate() that takes as input dataFrames and output dataFrames
    I think the wrapping part works as expected, however that does not mean that rotation
    is working. That still needs to be checked explicitly by comparing to strain computed
    on roated particle positions. 
    Also note that this wrapper would need some smalal modifications to work on the asymetric
    part of the strain tensor...mostly the signature needs to be modified. 
    Feb 6 2021
    """
    
    # take the strain df and take only strain components in the required order and convert to numpy array
    # do this using signature and compute sigList as above.
    if signature is None or signature == 'upper triangle':
        _sigList = ['exx','exy','exz','eyy','eyz','ezz']
        _sig_np = np.array([(i,j) for i in range(3) for j in range(i,3)])
    else: raise ValueError('signature {} is not supported. Use \'upper triangle\' '.format(signature))
    
    
    # numpy array of the strains, and index
    strainList = strain_df.loc[(slice(None),_sigList),time_str].to_numpy()
    idx = strain_df.loc[(slice(None),_sigList),time_str].index
    
    # split strainList, particle is going to be index on slowest axis
    Nparticle = strainList.shape[0]/len(_sigList)
    strainList = np.array(np.split(strainList, Nparticle))
    
    # get the rotation matrix
    #with open('rotation.yaml') as f: rotDict = yaml.load(f, Loader=yaml.FullLoader)
    #r = parseRotYaml(rotDict['rotationMatrix'])['prod_zyx (left)'] 
    
    strainList_rot = _rotate(strainList,_sig_np, r)
    
    # reformat into dataFrame to match input
    strain_rot_df = pd.DataFrame(data=strainList_rot.flatten(), index=idx, columns=[time_str])
    
    return strain_rot_df

In [283]:
tmp = np.array(np.split(strain_traj.loc[([80942,74955],_sigList),'(0,1)'].to_numpy(),2))
sig_np = np.array([(i,j) for i in range(3) for j in range(i,3)])
strain_rot = np.zeros((3,3))

In [304]:
strain_micro_df = strain_traj.loc[[80942,74955]]
rotateStrain(strain_micro_df,'(0,1)',r_left)

(0,1)
particle values          
80942    exx     0.001357
         exy    -0.001108
         exz     0.003032
         eyy    -0.000124
         eyz    -0.002062
         ezz    -0.006647
74955    exx     0.005999
         exy     0.001768
         exz    -0.003469
         eyy    -0.004522
         eyz    -0.020199
         ezz    -0.041497

In [341]:
#strain_traj.loc[[80942,74955]]
# computeStrain(pos_df, tPair, output = 'strainTraj', pos_keys=None, verbose=False):
pos_rot = rotatePosition(pos,r_left)
pos_rot

x (um, rheo_sedHeight)  y (um, rheo_sedHeight)  \
frame particle                                                   
0     74955                 -40.757087              -26.974613   
      74956                 -39.418112              -26.273026   
      75057                 -41.955359              -26.016520   
      75063                 -41.251155              -23.791074   
      75068                 -40.586414              -25.305694   
...                                ...                     ...   
2     976684                 22.382368               20.883370   
      976727                 30.897252               31.037122   
      976749                 27.574518               27.059527   
      976750                 28.118515               28.308266   
      976776                 27.609624               25.686123   

                z (um, rheo_sedHeight)  
frame particle                          
0     74955                  27.902609  
      74956                  27.900874  
      75057                  28.032157  
      75063                  27.864490  
      75068                  27.942663  
...                                ...  
2     976684                 83.545205  
      976727                 83.414681  
      976749                 84.499723  
      976750                 84.431944  
      976776                 85.342912  

[470123 rows x 3 columns]

In [342]:
_keys = {x:'{} (um, rheo_sedHeight)'.format(x) for x in ['x','y','z']}
strain_rotPos = computeStrain(pos_rot,(0,1),pos_keys = _keys)

In [344]:
strainTraj_rheoSedDept = computeStrain(pos,(0,1),pos_keys=rot_keys)

In [356]:
strain_rot = rotateStrain(strainTraj_rheoSedDept,'(0,1)',r_left)
strain_rot

(0,1)
particle values          
74955    exx     0.005993
         exy    -0.001785
         exz     0.003437
         eyy    -0.004516
         eyz    -0.020204
...                   ...
890420   exy    -0.000006
         exz     0.026016
         eyy     0.001207
         eyz     0.022468
         ezz     0.004417

[934416 rows x 1 columns]

In [355]:
r2 = r_left @ np.array([[1,0,0],[0,-1,0],[0,0,1]])
strain_rot2 = rotateStrain(strain_traj, '(0,1)',r2)
#strain_traj
strain_rot2

(0,1)
particle values          
74955    exx     0.005993
         exy    -0.001785
         exz     0.003437
         eyy    -0.004516
         eyz    -0.020204
...                   ...
890420   exy    -0.000006
         exz     0.026016
         eyy     0.001207
         eyz     0.022468
         ezz     0.004417

[934416 rows x 1 columns]

In [347]:
# I think this is is...
# the problem I had before with coordinates being flipped and everything be a little be off
# was a direct consequence of using almost 90 deg rotation and not computing the strain orginally on the
# rheosedDepth riight handed coordinate system. Instead I was using image coordinates and the strain magnitudes were correct
# but the signs were wrong. 
strain_rot, strain_rotPos.loc[(slice(None),['exx','exy','exz','eyy','eyz','ezz']),'(0,1)']

In [348]:
strainTraj_rheoSedDept.loc[(slice(None),['exx','exy','exz','eyy','eyz','ezz']),'(0,1)']

particle  values
74955     exx      -0.004519
          exy       0.001776
          exz      -0.020201
          eyy       0.005996
          eyz      -0.003453
                      ...   
890420    exy       0.000007
          exz       0.022489
          eyy      -0.000981
          eyz      -0.025998
          ezz       0.004417
Name: (0,1), Length: 934416, dtype: float64

In [329]:
-0.004519 + 0.005996

0.001477

In [331]:
0.005993-0.004516

0.001477

In [332]:
0.005999 - 0.004522

0.001477

## Tasks
- put rotation code into pycharm
- use rotation code to compute the parallel and perpendicular strains in tfr experiment for paper
- update figures with parallel and perp measurements. 

In [ ]:
# how to convert between these strain measures? 
# It should be conjugate with rotation matrix e_rot = R e R^T as in mohr circle

In [ ]:
# how does D2min work in this case? Should it stay unchanged? I am not sure. 

In [ ]:
# put the particle positions in the gel and sample in the same orthonormal basis treating the gel positions as say
# segative particle positions and compute relative displacement fo the boundaries. 

In [ ]:
# diagonalize the strain tensor and find component in shear and perp direction

In [ ]:
# revisit local von mises